In [378]:
import numpy as np
import pandas as pd
import joblib as jb
import math
import statistics
import scipy.stats
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr
alpha = 0.05

# DATA LOADING

In [379]:
dataset = "nyc-rolling-sales.csv"
nyc = pd.read_csv(dataset)

In [380]:
nyc

,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,,C2,153 AVENUE B,...,5,0,5,1633,6440,1900,2,C2,6625000,2017-07-19 00:00:00
1,5,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,26,,C7,234 EAST 4TH STREET,...,28,3,31,4616,18690,1900,2,C7,-,2016-12-14 00:00:00
2,6,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,39,,C7,197 EAST 3RD STREET,...,16,1,17,2212,7803,1900,2,C7,-,2016-12-09 00:00:00
3,7,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,,C4,154 EAST 7TH STREET,...,10,0,10,2272,6794,1913,2,C4,3936272,2016-09-23 00:00:00
4,8,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,,C2,301 EAST 10TH STREET,...,6,0,6,2369,4615,1900,2,C2,8000000,2016-11-17 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84543,8409,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7349,34,,B9,37 QUAIL LANE,...,2,0,2,2400,2575,1998,1,B9,450000,2016-11-28 00:00:00
84544,8410,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7349,78,,B9,32 PHEASANT LANE,...,2,0,2,2498,2377,1998,1,B9,550000,2017-04-21 00:00:00
84545,8411,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7351,60,,B2,49 PITNEY AVENUE,...,2,0,2,4000,1496,1925,1,B2,460000,2017-07-05 00:00:00
84546,8412,5,WOODROW,22 STORE BUILDINGS,4,7100,28,,K6,2730 ARTHUR KILL ROAD,...,0,7,7,208033,64117,2001,4,K6,11693337,2016-12-21 00:00:00


In [381]:
print(nyc.dtypes)

Unnamed: 0                         int64
BOROUGH                            int64
NEIGHBORHOOD                      object
BUILDING CLASS CATEGORY           object
TAX CLASS AT PRESENT              object
BLOCK                              int64
LOT                                int64
EASE-MENT                         object
BUILDING CLASS AT PRESENT         object
ADDRESS                           object
APARTMENT NUMBER                  object
ZIP CODE                           int64
RESIDENTIAL UNITS                  int64
COMMERCIAL UNITS                   int64
TOTAL UNITS                        int64
LAND SQUARE FEET                  object
GROSS SQUARE FEET                 object
YEAR BUILT                         int64
TAX CLASS AT TIME OF SALE          int64
BUILDING CLASS AT TIME OF SALE    object
SALE PRICE                        object
SALE DATE                         object
dtype: object


In [382]:
nyc.tail(10)

,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
84538,8404,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7316,61,,B2,178 DARNELL LANE,...,2,0,2,3215,1300,1995,1,B2,-,2017-06-30 00:00:00
84539,8405,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7316,85,,B2,137 DARNELL LANE,...,2,0,2,3016,1300,1995,1,B2,-,2016-12-30 00:00:00
84540,8406,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7316,93,,B2,125 DARNELL LANE,...,2,0,2,3325,1300,1995,1,B2,509000,2016-10-31 00:00:00
84541,8407,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7317,126,,B2,112 ROBIN COURT,...,2,0,2,11088,2160,1994,1,B2,648000,2016-12-07 00:00:00
84542,8408,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7339,41,,B9,41 SONIA COURT,...,2,0,2,3020,1800,1997,1,B9,-,2016-12-01 00:00:00
84543,8409,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7349,34,,B9,37 QUAIL LANE,...,2,0,2,2400,2575,1998,1,B9,450000,2016-11-28 00:00:00
84544,8410,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7349,78,,B9,32 PHEASANT LANE,...,2,0,2,2498,2377,1998,1,B9,550000,2017-04-21 00:00:00
84545,8411,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7351,60,,B2,49 PITNEY AVENUE,...,2,0,2,4000,1496,1925,1,B2,460000,2017-07-05 00:00:00
84546,8412,5,WOODROW,22 STORE BUILDINGS,4,7100,28,,K6,2730 ARTHUR KILL ROAD,...,0,7,7,208033,64117,2001,4,K6,11693337,2016-12-21 00:00:00
84547,8413,5,WOODROW,35 INDOOR PUBLIC AND CULTURAL FACILITIES,4,7105,679,,P9,155 CLAY PIT ROAD,...,0,1,1,10796,2400,2006,4,P9,69300,2016-10-27 00:00:00


In [383]:
x = np.array(nyc[['YEAR BUILT']])
y = np.array(nyc[['SALE PRICE']])

In [384]:
x

array([[1900],
       [1900],
       [1900],
       ...,
       [1925],
       [2001],
       [2006]], dtype=int64)

In [385]:
y

array([['6625000'],
       [' -  '],
       [' -  '],
       ...,
       ['460000'],
       ['11693337'],
       ['69300']], dtype=object)

In [386]:
x = nyc['YEAR BUILT']
x_with_nan = [1633, 6440, math.nan, 1900]
print(x)
print(x_with_nan)

0        1900
1        1900
2        1900
3        1913
4        1900
         ... 
84543    1998
84544    1998
84545    1925
84546    2001
84547    2006
Name: YEAR BUILT, Length: 84548, dtype: int64
[1633, 6440, nan, 1900]


In [387]:
y, y_with_nan = np.array(x), np.array(x_with_nan)
z, z_with_nan = pd.Series(x), pd.Series(x_with_nan)
print(y)
print(y_with_nan)
print(z_with_nan)

[1900 1900 1900 ... 1925 2001 2006]
[1633. 6440.   nan 1900.]
0    1633.0
1    6440.0
2       NaN
3    1900.0
dtype: float64


# ANALISA

data yang saya pakai adalah kumpulan catatan setiap bangunan atau unit bangunan (apartemen, dll.) yang dijual di pasar properti Kota New York selama periode 12 bulan. terdapat banyak daerah didalamnya. Pada kali ini saya akan mengulas data pada daerah WOODROW.
terdapat banyak kriteria data dalam penjualan. saya mengambil dua data yaitu luas kotak tanah (LAND SQUARE FEET) dan luas tanah yang terpakai (GROSS SQUARE FEET)   

# WOODROW

## Measures of Central Tendency

In [388]:

#Memilih tempat di woodrow
nyc_woodrow = nyc[nyc['NEIGHBORHOOD'] == 'WOODROW']
nyc_woodrow.head()

,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
84417,8283,5,WOODROW,01 ONE FAMILY DWELLINGS,1,6912,115,,A1,1842 DRUMGOOLE ROAD WEST,...,1,0,1,6400,2536,1980,1,A1,640000,2016-09-21 00:00:00
84418,8284,5,WOODROW,01 ONE FAMILY DWELLINGS,1,6914,69,,A1,563 DARLINGTON AVENUE,...,1,0,1,4000,2024,1974,1,A1,664000,2016-11-22 00:00:00
84419,8285,5,WOODROW,01 ONE FAMILY DWELLINGS,1,6918,38,,A2,966 IONIA AVENUE,...,1,0,1,12000,1826,1980,1,A2,-,2017-06-01 00:00:00
84420,8286,5,WOODROW,01 ONE FAMILY DWELLINGS,1,6919,105,,A3,1009 IONIA AVENUE,...,1,0,1,10000,4326,1985,1,A3,-,2017-04-27 00:00:00
84421,8287,5,WOODROW,01 ONE FAMILY DWELLINGS,1,6920,51,,A2,374 FOSTER ROAD,...,1,0,1,8000,2450,1975,1,A2,-,2017-02-03 00:00:00


In [389]:
nyc_woodrow[nyc_woodrow['RESIDENTIAL UNITS'] == 1]

,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
84417,8283,5,WOODROW,01 ONE FAMILY DWELLINGS,1,6912,115,,A1,1842 DRUMGOOLE ROAD WEST,...,1,0,1,6400,2536,1980,1,A1,640000,2016-09-21 00:00:00
84418,8284,5,WOODROW,01 ONE FAMILY DWELLINGS,1,6914,69,,A1,563 DARLINGTON AVENUE,...,1,0,1,4000,2024,1974,1,A1,664000,2016-11-22 00:00:00
84419,8285,5,WOODROW,01 ONE FAMILY DWELLINGS,1,6918,38,,A2,966 IONIA AVENUE,...,1,0,1,12000,1826,1980,1,A2,-,2017-06-01 00:00:00
84420,8286,5,WOODROW,01 ONE FAMILY DWELLINGS,1,6919,105,,A3,1009 IONIA AVENUE,...,1,0,1,10000,4326,1985,1,A3,-,2017-04-27 00:00:00
84421,8287,5,WOODROW,01 ONE FAMILY DWELLINGS,1,6920,51,,A2,374 FOSTER ROAD,...,1,0,1,8000,2450,1975,1,A2,-,2017-02-03 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84494,8360,5,WOODROW,01 ONE FAMILY DWELLINGS,1,7351,4,,A5,241 GAYNOR STREET,...,1,0,1,2500,1500,2000,1,A5,517500,2016-09-16 00:00:00
84495,8361,5,WOODROW,01 ONE FAMILY DWELLINGS,1,7355,23,,A1,79 CHURCHILL AVENUE,...,1,0,1,4628,1680,2001,1,A1,774143,2017-06-29 00:00:00
84496,8362,5,WOODROW,01 ONE FAMILY DWELLINGS,1,7355,70,,A5,68 AMBASSADOR LANE,...,1,0,1,2526,2160,2001,1,A5,523950,2016-10-24 00:00:00
84497,8363,5,WOODROW,01 ONE FAMILY DWELLINGS,1,7355,95,,A5,115 AMBASSADOR LANE,...,1,0,1,8846,1710,2001,1,A5,605000,2017-02-15 00:00:00


In [390]:
nyc_woodrow['LAND SQUARE FEET'] = pd.to_numeric(nyc_woodrow['LAND SQUARE FEET'])
nyc_woodrow['LAND SQUARE FEET']

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1308\626346057.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nyc_woodrow['LAND SQUARE FEET'] = pd.to_numeric(nyc_woodrow['LAND SQUARE FEET'])


84417      6400
84418      4000
84419     12000
84420     10000
84421      8000
          ...  
84543      2400
84544      2498
84545      4000
84546    208033
84547     10796
Name: LAND SQUARE FEET, Length: 131, dtype: int64

In [391]:
nyc_woodrow['GROSS SQUARE FEET'] = pd.to_numeric(nyc_woodrow['GROSS SQUARE FEET'])
nyc_woodrow['GROSS SQUARE FEET']

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_1308\1002682042.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nyc_woodrow['GROSS SQUARE FEET'] = pd.to_numeric(nyc_woodrow['GROSS SQUARE FEET'])


84417     2536
84418     2024
84419     1826
84420     4326
84421     2450
         ...  
84543     2575
84544     2377
84545     1496
84546    64117
84547     2400
Name: GROSS SQUARE FEET, Length: 131, dtype: int64

In [392]:
nyc_woodrow_building = nyc_woodrow[(nyc_woodrow['RESIDENTIAL UNITS'] == 1) & 
           (nyc_woodrow['TOTAL UNITS'] == 1) & 
           (~nyc_woodrow['GROSS SQUARE FEET'].isna()) & 
           (~nyc_woodrow['LAND SQUARE FEET'].isna())]

### Mean

##### Mencari rata rata pada 'GROSS SQUARE FEET' DAN 'LAND SQUARE FEET' di WOODROW

In [393]:
print(nyc_woodrow_building['GROSS SQUARE FEET'].mean())
print(nyc_woodrow_building['LAND SQUARE FEET'].mean())

1758.1463414634147
3282.878048780488


In [394]:
x_woodrow = np.array(nyc_woodrow[['GROSS SQUARE FEET']])
y_woodrow = np.array(nyc_woodrow[['LAND SQUARE FEET']])

In [395]:
y_woodrow

array([[  6400],
       [  4000],
       [ 12000],
       [ 10000],
       [  8000],
       [  2000],
       [  1900],
       [  1900],
       [  1900],
       [  3553],
       [  3553],
       [  3857],
       [  6837],
       [  6000],
       [  4753],
       [  5440],
       [  4000],
       [  5000],
       [   770],
       [  1080],
       [  2565],
       [  2565],
       [  2565],
       [  2850],
       [  2500],
       [  4250],
       [  2561],
       [  2773],
       [  2773],
       [ 10000],
       [  4000],
       [  4092],
       [  2133],
       [  4182],
       [  2645],
       [  2704],
       [   707],
       [  1293],
       [ 13253],
       [  3379],
       [  2365],
       [  2077],
       [  2464],
       [  2464],
       [ 10020],
       [  1292],
       [  1130],
       [  1312],
       [  1062],
       [  2229],
       [  1062],
       [  1062],
       [  1376],
       [  1240],
       [  1137],
       [  1137],
       [  1328],
       [  1328],
       [  1110

In [396]:
x_woodrow

array([[ 2536],
       [ 2024],
       [ 1826],
       [ 4326],
       [ 2450],
       [ 1496],
       [ 1496],
       [ 1496],
       [ 1496],
       [ 2166],
       [ 2166],
       [ 1040],
       [ 2377],
       [ 3012],
       [ 2244],
       [ 2350],
       [ 1976],
       [  840],
       [  960],
       [  960],
       [ 1600],
       [ 1600],
       [ 1600],
       [ 1600],
       [ 1240],
       [ 1650],
       [ 1592],
       [ 1650],
       [ 1650],
       [ 6000],
       [ 1196],
       [ 1196],
       [ 2079],
       [ 1032],
       [ 1600],
       [ 1600],
       [ 1900],
       [ 1900],
       [ 4000],
       [ 1439],
       [ 1432],
       [ 1848],
       [ 1860],
       [ 1860],
       [ 3200],
       [ 1032],
       [  977],
       [ 1570],
       [ 1032],
       [ 1032],
       [ 1570],
       [ 1032],
       [ 1570],
       [ 1435],
       [ 1435],
       [ 1435],
       [ 1080],
       [ 1570],
       [ 1050],
       [ 1050],
       [ 1300],
       [ 1300],
       [

In [397]:
# rata rata dari Land square feet
mean = sum(y_woodrow) / len(y_woodrow)
mean

array([5678.79389313])

In [398]:
#rata rata gross square feet
mean = sum(x_woodrow) / len(x_woodrow)
mean

array([2445.29770992])

##### mean GROSS SQUARE FEET 1758.1463414634147
##### men LAND SQUARE FEET 3282.878048780488

## MEDIAN

In [399]:
#Menghitung median unit total di woodrow
print(nyc_woodrow_building['GROSS SQUARE FEET'].median())
print(nyc_woodrow_building['LAND SQUARE FEET'].median())

1600.0
2563.0


#### Mencari nilai tengah pada 'GROSS SQUARE FEET = 1600' DAN 'LAND SQUARE FEET= 2563' di WOODROW

## MODE

In [424]:
print(nyc_woodrow_building['GROSS SQUARE FEET'].mode())
print(nyc_woodrow_building['LAND SQUARE FEET'].mode())

0    1600
Name: GROSS SQUARE FEET, dtype: int64
0    4000
Name: LAND SQUARE FEET, dtype: int64


##### Mencari nilai frekuensi terbesar pada 'GROSS SQUARE FEET= 1600' DAN 'LAND SQUARE FEET= 4000' di WOODROW

## RANGE

In [401]:
print(np.ptp(nyc_woodrow_building['GROSS SQUARE FEET'])) 
print(np.ptp(nyc_woodrow_building['LAND SQUARE FEET'])) 

5160
12546


##### Mencari jangkauan pada 'GROSS SQUARE FEET = 5160' DAN 'LAND SQUARE FEET= 12546' di WOODROW

## VARIANCE

In [402]:
print(nyc_woodrow_building['GROSS SQUARE FEET'].var())
print(nyc_woodrow_building['LAND SQUARE FEET'].var())

581125.1388136102
6579728.207166515


##### Mencari perbedaan pada 'GROSS SQUARE FEET = 5181125' DAN 'LAND SQUARE FEET = 6579728' di WOODROW

## Standard Deviation

In [403]:
print(statistics.stdev(nyc_woodrow_building['GROSS SQUARE FEET']))
print(statistics.stdev(nyc_woodrow_building['LAND SQUARE FEET']))

762.3156425088039
2565.0980891900635


#### Mencari standar deviasi pada 'GROSS SQUARE FEET = 762' DAN 'LAND SQUARE FEET = 2565' di WOODROW

## PROBABILITAS DISTRIBUSI

In [404]:
ax = sns.displot(nyc_woodrow_building['GROSS SQUARE FEET'],
                 kde=True,
                 bins=100,
                )
ax.set_titles('Probability Distribution for Gross Square Feet in Woodrow')
ax.set_ylabels('Frequency')

In [406]:
ax = sns.displot(nyc_woodrow_building['LAND SQUARE FEET'],
                 kde=True,
                 bins=100,
                )
ax.set_titles('Probability Distribution for land Square Feet in Astoria')
ax.set_ylabels('Frequency')

##  Median dari Tahun Pembangunan

In [407]:
X = np.array(nyc_woodrow[['YEAR BUILT']])

In [408]:
#NILAI TENGAH TAHUN DARI PEMBANGUNAN
sorted(X)

[array([1925], dtype=int64),
 array([1950], dtype=int64),
 array([1950], dtype=int64),
 array([1955], dtype=int64),
 array([1970], dtype=int64),
 array([1970], dtype=int64),
 array([1974], dtype=int64),
 array([1975], dtype=int64),
 array([1975], dtype=int64),
 array([1975], dtype=int64),
 array([1976], dtype=int64),
 array([1976], dtype=int64),
 array([1979], dtype=int64),
 array([1980], dtype=int64),
 array([1980], dtype=int64),
 array([1980], dtype=int64),
 array([1980], dtype=int64),
 array([1980], dtype=int64),
 array([1980], dtype=int64),
 array([1981], dtype=int64),
 array([1982], dtype=int64),
 array([1984], dtype=int64),
 array([1984], dtype=int64),
 array([1984], dtype=int64),
 array([1984], dtype=int64),
 array([1984], dtype=int64),
 array([1984], dtype=int64),
 array([1984], dtype=int64),
 array([1984], dtype=int64),
 array([1985], dtype=int64),
 array([1985], dtype=int64),
 array([1985], dtype=int64),
 array([1985], dtype=int64),
 array([1985], dtype=int64),
 array([1985],

In [409]:
n = len(X)
if n % 2 != 0:
    median_ = sorted(X)[round(0.5*(n-1))]
else:
    x_ord, index = sorted(X), round(0.5 * n)
    median_ = 0.5 * (x_ord[index-1] + x_ord[index])

median_

array([1992], dtype=int64)

In [410]:
X

array([[1980],
       [1974],
       [1980],
       [1985],
       [1975],
       [1984],
       [1985],
       [1985],
       [1985],
       [1984],
       [1984],
       [1950],
       [1980],
       [2005],
       [1981],
       [1975],
       [1975],
       [1955],
       [1987],
       [1987],
       [2002],
       [2002],
       [2002],
       [2002],
       [1987],
       [1999],
       [1998],
       [1999],
       [1999],
       [1984],
       [1970],
       [1970],
       [1980],
       [1980],
       [2001],
       [2001],
       [1992],
       [1992],
       [1997],
       [2000],
       [2011],
       [2012],
       [2011],
       [2011],
       [1997],
       [1991],
       [1991],
       [1991],
       [1992],
       [1992],
       [1992],
       [1992],
       [1993],
       [1990],
       [1990],
       [1990],
       [1990],
       [1990],
       [1990],
       [1990],
       [1996],
       [1995],
       [1995],
       [1995],
       [1995],
       [1997],
       [19

In [411]:
print(statistics.median(X[:-1]))
print(statistics.median_low(X[:-1]))
print(statistics.median_high(X[:-1]))

[1992.]
[1992]
[1992]


In [412]:
median_ = np.median(X)
print(median_)

1992.0


#### median dari tahun dibangunnya adalah tahun 1992

# Measures of Variability

#### rata rata dari tahun dibangunnya bangunan adalah 1991

In [413]:
print(sorted(X))
print(mean_)

[array([1925], dtype=int64), array([1950], dtype=int64), array([1950], dtype=int64), array([1955], dtype=int64), array([1970], dtype=int64), array([1970], dtype=int64), array([1974], dtype=int64), array([1975], dtype=int64), array([1975], dtype=int64), array([1975], dtype=int64), array([1976], dtype=int64), array([1976], dtype=int64), array([1979], dtype=int64), array([1980], dtype=int64), array([1980], dtype=int64), array([1980], dtype=int64), array([1980], dtype=int64), array([1980], dtype=int64), array([1980], dtype=int64), array([1981], dtype=int64), array([1982], dtype=int64), array([1984], dtype=int64), array([1984], dtype=int64), array([1984], dtype=int64), array([1984], dtype=int64), array([1984], dtype=int64), array([1984], dtype=int64), array([1984], dtype=int64), array([1984], dtype=int64), array([1985], dtype=int64), array([1985], dtype=int64), array([1985], dtype=int64), array([1985], dtype=int64), array([1985], dtype=int64), array([1985], dtype=int64), array([1985], dtype

In [414]:
n = len(X)

mean_ = sum(X) / n

var_ = sum((item - mean_)**2 for item in X) / (n - 1)
var_

array([154.15795655])

In [415]:
var_ = np.var(X, ddof=1)
var_

154.1579565472695

In [416]:
X.var(ddof=1)

154.1579565472695

## Confidence Intervals

In [419]:
# df_astoria_one_resident[df_astoria_one_resident['BUILDING CLASS AT PRESENT'] == 'A2']
x = nyc_woodrow['BUILDING CLASS AT PRESENT'].unique()
y = nyc_woodrow_building['BUILDING CLASS AT PRESENT'].unique()
print(x, y, sep='\n')

['A1' 'A2' 'A3' 'A5' 'B2' 'B9' 'K6' 'P9']
['A1' 'A2' 'A3' 'A5']


In [420]:
mean_aor = nyc_woodrow_building['LAND SQUARE FEET'].mean()
n_aor = nyc_woodrow_building['LAND SQUARE FEET'].count()
sd_aor = statistics.stdev(nyc_woodrow_building['LAND SQUARE FEET'])
z = 1.96 #95%
sd_aor

2565.0980891900635

In [421]:
lcb = mean_aor - z * (sd_aor/np.sqrt(n_aor))
ucb = mean_aor + z * (sd_aor/np.sqrt(n_aor))

lcb, ucb

(2727.6733635994624, 3838.0827339615134)

##### nilai parameter sebuah 'LAND SQUARE FEET' terletak di dalam interval 2727.6733635994624 - 3838.0827339615134

In [422]:
mean_aor = nyc_woodrow_building['GROSS SQUARE FEET'].mean()
n_aor = nyc_woodrow_building['GROSS SQUARE FEET'].count()
sd_aor = statistics.stdev(nyc_woodrow_building['GROSS SQUARE FEET'])
z = 1.96 #95%
sd_aor

762.3156425088039

In [423]:
lcb = mean_aor - z * (sd_aor/np.sqrt(n_aor))
ucb = mean_aor + z * (sd_aor/np.sqrt(n_aor))

lcb, ucb

(1593.146329147585, 1923.1463537792445)

##### nilai parameter sebuah 'GROSS SQUARE FEET' terletak di dalam interval 1593.146329147585 - 1923.1463537792445

# Hypothesis Testing

In [377]:
stat, p = pearsonr(nyc_woodrow_building['LAND SQUARE FEET'], nyc_woodrow_building['GROSS SQUARE FEET'])
print('stat: ', stat , ' p: ' ,p)
if p > alpha:
    print('independen')
else:
    print('dependen')

stat:  0.6793687891838068  p:  2.2752484752414272e-12
dependen


#### maka data yang di bahas terdeteksi variabel yang tergantung pada faktor-faktor lain